In [ ]:
import pandas as pd

import oversampling_binary
from oversampling_binary import split_dataset, apply_smote
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Caricare il dataset e applicare lo split
csv_file_path = 'audio_features.csv'

In [ ]:
# Carica il file CSV in un DataFrame
df = pd.read_csv(csv_file_path)

# Filtra solo i dati con Class = Target
df_target = df[df['Class'] == 'Target']

# Conta il numero di occorrenze per ciascuna subclass
subclass_counts = df_target['Subclass'].value_counts()

# Stampa ogni subclass con il numero di elementi
for subclass, count in subclass_counts.items():
    print(f"{subclass}: {count}")

# Stampa il numero totale di elementi
total_elements = subclass_counts.sum()
print(f"Numero totale di elementi (Class = Target): {total_elements}")

In [ ]:

# Filtra solo i dati con Class = Target
df_no_target = df[df['Class'] == 'Non-Target']

# Conta il numero di occorrenze per ciascuna subclass
subclass_counts_no = df_no_target['Subclass'].value_counts()

# Stampa ogni subclass con il numero di elementi
for subclass, count in subclass_counts_no.items():
    print(f"{subclass}: {count}")

# Stampa il numero totale di elementi
total_elements_no = subclass_counts_no.sum()
print(f"Numero totale di elementi (Class = Non Target): {total_elements_no}")

# SPLIT

In [ ]:
X_train_imputed, X_val_imputed, X_test_imputed, y_train_encoded, y_val_encoded, y_test_encoded, subclasses_train, class_encoder = split_dataset('audio_features.csv')


## SMOTE

In [ ]:
X_train_resampled, y_train_resampled = apply_smote(X_train_imputed, y_train_encoded, subclasses_train)

In [ ]:
print("Classi nel set di addestramento:", np.unique(y_train_resampled))
print("Classi nel set di validazione:", np.unique(y_val_encoded))
print("Classi nel set di test:", np.unique(y_test_encoded))

In [ ]:
# Imposta pandas per visualizzare tutto il contenuto
pd.set_option('display.max_rows', None)  # Rimuove la limitazione del numero massimo di righe visualizzate
pd.set_option('display.max_columns', None)  # Mostra tutte le colonne

## Distribuzione delle classi nel set di addestramento dopo SMOTE

In [ ]:
y_train_resampled_df = pd.DataFrame(y_train_resampled, columns=['Class'])
print("Distribuzione delle classi nel set di training dopo SMOTE:")
display(y_train_resampled_df['Class'].value_counts())

# ESPERIMENTO 1

Esperimento effettuato prendendo come riferimento alcune features numeriche da "Malfante et al."

## ADDESTRAMENTO RANDOM FOREST

In [ ]:
# Converte i DataFrame in ndarray
X_train_resampled_array = X_train_resampled  # Supponendo che sia già un ndarray
X_val_imputed_array = X_val_imputed.values  # Converti il DataFrame in ndarray
X_test_imputed_array = X_test_imputed.values  # Converti il DataFrame in ndarray

In [ ]:
# Addestra il modello Random Forest
random_forest_model = oversampling_binary.train_random_forest(
    X_train_resampled_array,
    y_train_resampled,
    X_val_imputed_array,
    y_val_encoded,
    X_test_imputed_array,
    y_test_encoded
)

In [ ]:
# 1. Effettua le predizioni sul set di convalida
y_pred_val = random_forest_model.predict(X_val_imputed_array)

# 2. Effettua le predizioni sul set di test
y_pred_test = random_forest_model.predict(X_test_imputed_array)

# 3. Calcola le matrici di confusione per convalida e test
cm_val = confusion_matrix(y_val_encoded, y_pred_val)
cm_test = confusion_matrix(y_test_encoded, y_pred_test)

# 4. Visualizza entrambe le matrici di confusione
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Matrice di confusione per il set di convalida
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[0])
axes[0].set_title('Random Forest - Matrice di Confusione (Validazione)')
axes[0].set_xlabel('Predizioni')
axes[0].set_ylabel('Label')

# Matrice di confusione per il set di test
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[1])
axes[1].set_title('Random Forest - Matrice di Confusione (Test)')
axes[1].set_xlabel('Predizioni')
axes[1].set_ylabel('Label')


plt.tight_layout()
plt.show()

# ADDESTRAMENTO SVM

In [ ]:
svm_model = oversampling_binary.train_svm(X_train_resampled_array, y_train_resampled, X_val_imputed_array, y_val_encoded, X_test_imputed_array, y_test_encoded)

In [ ]:
# 1. Effettua le predizioni sul set di convalida
y_pred_val = svm_model.predict(X_val_imputed_array)

# 2. Effettua le predizioni sul set di test
y_pred_test = svm_model.predict(X_test_imputed_array)

# 3. Calcola le matrici di confusione per convalida e test
cm_val = confusion_matrix(y_val_encoded, y_pred_val)
cm_test = confusion_matrix(y_test_encoded, y_pred_test)

# 4. Visualizza entrambe le matrici di confusione
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Matrice di confusione per il set di convalida
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[0])
axes[0].set_title('SVM - Matrice di Confusione (Validazione)')
axes[0].set_xlabel('Predizioni')
axes[0].set_ylabel('Label')

# Matrice di confusione per il set di test
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[1])
axes[1].set_title('SVM - Matrice di Confusione (Test)')
axes[1].set_xlabel('Predizioni')
axes[1].set_ylabel('Label')


plt.tight_layout()
plt.show()

# ADDESTRAMENTO LIGHT GBM

In [ ]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"  # Adatta questo valore al numero di core che desideri utilizzare

In [ ]:
y_train_resampled = y_train_resampled.astype(int)
y_val_encoded = y_val_encoded.astype(int)
y_test_encoded = y_test_encoded.astype(int)


In [ ]:
lightgbm_model = oversampling_binary.train_lightgbm(X_train_resampled_array, y_train_resampled, X_val_imputed_array, y_val_encoded, X_test_imputed_array, y_test_encoded)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred_proba = lightgbm_model.predict(X_val_imputed_array)
# Converti le probabilità in etichette di classe
y_val_pred = np.argmax(y_val_pred_proba, axis=1)

# Predizioni del modello per il set di test
y_test_pred_proba = lightgbm_model.predict(X_test_imputed_array)
# Converti le probabilità in etichette di classe
y_test_pred = np.argmax(y_test_pred_proba, axis=1)

# Genera la matrice di confusione per il set di validazione
cm_val = confusion_matrix(y_val_encoded, y_val_pred, labels=np.unique(y_val_encoded))
disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=np.unique(y_val_encoded))

# Genera la matrice di confusione per il set di test
cm_test = confusion_matrix(y_test_encoded, y_test_pred, labels=np.unique(y_test_encoded))
disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=np.unique(y_test_encoded))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("LightGBM - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("LightGBM - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()

# Esperimento 2

Esperimento effettuato prendendo come riferimento alcune features numeriche da "BirdNet"

In [ ]:
#SPLIT
X_train_imputed_exp1, X_val_imputed_exp1, X_test_imputed_exp1, y_train_encoded_filtered_exp1, y_val_encoded_exp1, y_test_encoded_exp1, subclasses_train, class_encoder_exp1 = split_dataset(
    'exp1_audio_features.csv')

In [ ]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"  # Adatta questo valore al numero di core che desideri utilizzare

In [ ]:
#SMOTE
X_train_resampled_exp1, y_train_resampled_exp1 = apply_smote(X_train_imputed_exp1, y_train_encoded_filtered_exp1, subclasses_train)

In [ ]:
y_train_resampled_df_exp1 = pd.DataFrame(y_train_resampled_exp1, columns=['Class'])
print("Distribuzione delle classi nel set di training dopo SMOTE:")
display(y_train_resampled_df_exp1['Class'].value_counts())

In [ ]:
# Converte i DataFrame in ndarray
X_train_resampled_array_exp1 = X_train_resampled_exp1  # Supponendo che sia già un ndarray
X_val_imputed_array_exp1 = X_val_imputed_exp1.values  # Converti il DataFrame in ndarray
X_test_imputed_array_exp1 = X_test_imputed_exp1.values  # Converti il DataFrame in ndarray

In [ ]:
#TRAIN
random_forest_model_exp1 = oversampling_binary.train_random_forest(X_train_resampled_exp1, y_train_resampled_exp1, X_val_imputed_array_exp1, y_val_encoded_exp1, X_test_imputed_array_exp1, y_test_encoded_exp1)

In [ ]:
# 1. Effettua le predizioni sul set di convalida
y_pred_val_exp1 = random_forest_model_exp1.predict(X_val_imputed_array_exp1)

# 2. Effettua le predizioni sul set di test
y_pred_test_exp1 = random_forest_model_exp1.predict(X_test_imputed_array_exp1)

# 3. Calcola le matrici di confusione per convalida e test
cm_val_exp1 = confusion_matrix(y_val_encoded_exp1, y_pred_val_exp1)
cm_test_exp1 = confusion_matrix(y_test_encoded_exp1, y_pred_test_exp1)

# 4. Visualizza entrambe le matrici di confusione
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Matrice di confusione per il set di convalida
sns.heatmap(cm_val_exp1, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[0])
axes[0].set_title('Random Forest - Matrice di Confusione (Validazione)')
axes[0].set_xlabel('Predizioni')
axes[0].set_ylabel('Label')

# Matrice di confusione per il set di test
sns.heatmap(cm_test_exp1, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[1])
axes[1].set_title('Random Forest - Matrice di Confusione (Test)')
axes[1].set_xlabel('Predizioni')
axes[1].set_ylabel('Label')


plt.tight_layout()
plt.show()

In [ ]:
svm_model_exp1 = oversampling_binary.train_svm(X_train_resampled_exp1, y_train_resampled_exp1, X_val_imputed_array_exp1, y_val_encoded_exp1, X_test_imputed_array_exp1, y_test_encoded_exp1)

In [ ]:
# 1. Effettua le predizioni sul set di convalida
y_pred_val_exp1 = svm_model_exp1.predict(X_val_imputed_array_exp1)

# 2. Effettua le predizioni sul set di test
y_pred_test_exp1 = svm_model_exp1.predict(X_test_imputed_array_exp1)

# 3. Calcola le matrici di confusione per convalida e test
cm_val_exp1 = confusion_matrix(y_val_encoded_exp1, y_pred_val_exp1)
cm_test_exp1 = confusion_matrix(y_test_encoded_exp1, y_pred_test_exp1)

# 4. Visualizza entrambe le matrici di confusione
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Matrice di confusione per il set di convalida
sns.heatmap(cm_val_exp1, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[0])
axes[0].set_title('SVM - Matrice di Confusione (Validazione)')
axes[0].set_xlabel('Predizioni')
axes[0].set_ylabel('Label')

# Matrice di confusione per il set di test
sns.heatmap(cm_test_exp1, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[1])
axes[1].set_title('SVM - Matrice di Confusione (Test)')
axes[1].set_xlabel('Predizioni')
axes[1].set_ylabel('Label')


plt.tight_layout()
plt.show()

In [ ]:
y_train_resampled_exp1 = y_train_resampled_exp1.astype(int)
y_val_encoded_exp1 = y_val_encoded_exp1.astype(int)
y_test_encoded_exp1 = y_test_encoded_exp1.astype(int)

In [ ]:
lightgbm_model_exp1 = oversampling_binary.train_lightgbm(
    X_train_resampled_array_exp1,
    y_train_resampled_exp1,
    X_val_imputed_array_exp1,
    y_val_encoded_exp1,
    X_test_imputed_array_exp1,  
    y_test_encoded_exp1
)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred_proba_exp1 = lightgbm_model_exp1.predict(X_val_imputed_array_exp1)
# Converti le probabilità in etichette di classe
y_val_pred_exp1 = np.argmax(y_val_pred_proba_exp1, axis=1)

# Predizioni del modello per il set di test
y_test_pred_proba_exp1 = lightgbm_model_exp1.predict(X_test_imputed_array_exp1)
# Converti le probabilità in etichette di classe
y_test_pred_exp1 = np.argmax(y_test_pred_proba_exp1, axis=1)

# Genera la matrice di confusione per il set di validazione
cm_val_exp1 = confusion_matrix(y_val_encoded_exp1, y_val_pred_exp1, labels=np.unique(y_val_encoded_exp1))
disp_val_exp1 = ConfusionMatrixDisplay(confusion_matrix=cm_val_exp1, display_labels=np.unique(y_val_encoded_exp1))

# Genera la matrice di confusione per il set di test
cm_test_exp1 = confusion_matrix(y_test_encoded_exp1, y_test_pred_exp1, labels=np.unique(y_test_encoded_exp1))
disp_test_exp1 = ConfusionMatrixDisplay(confusion_matrix=cm_test_exp1, display_labels=np.unique(y_test_encoded_exp1))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("LightGBM - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("LightGBM - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()

# ESPERIMENTO 3

Esperimento effettuato prendendo come riferimento features numeriche da "A Survey on Audio Feature Extraction for Automatic Music Genre Classification" di Dhamodaran et al.

In [ ]:
#SPLIT
X_train_imputed_exp2, X_val_imputed_exp2, X_test_imputed_exp2, y_train_encoded_filtered_exp2, y_val_encoded_exp2, y_test_encoded_exp2, subclasses_train, class_encoder_exp2 = split_dataset(
    'exp2_audio_features.csv')

In [ ]:
#SMOTE
X_train_resampled_exp2, y_train_resampled_exp2 = apply_smote(X_train_imputed_exp2, y_train_encoded_filtered_exp2, subclasses_train)

pd.set_option('display.max_rows', None)  # Rimuove la limitazione del numero massimo di righe visualizzate
pd.set_option('display.max_columns', None)  # Mostra tutte le colonne

In [ ]:
# Converti i dati in DataFrame
X_train_df_exp2 = pd.DataFrame(X_train_resampled_exp2)
X_val_df_exp2 = pd.DataFrame(X_val_imputed_exp2)
X_test_df_exp2 = pd.DataFrame(X_test_imputed_exp2)
# Aggiungi le etichette per il target
X_train_df_exp2['Class'] = y_train_resampled_exp2
X_val_df_exp2['Class'] = y_val_encoded_exp2
X_test_df_exp2['Class'] = y_test_encoded_exp2

In [ ]:
y_train_resampled_df_exp2 = pd.DataFrame(y_train_resampled_exp2, columns=['Class'])
print("Distribuzione delle classi nel set di training dopo SMOTE:")
display(y_train_resampled_df_exp2['Class'].value_counts())

In [ ]:
# Converte i DataFrame in ndarray
X_train_resampled_array_exp2 = X_train_resampled_exp2  # Supponendo che sia già un ndarray
X_val_imputed_array_exp2 = X_val_imputed_exp2.values  # Converti il DataFrame in ndarray
X_test_imputed_array_exp2 = X_test_imputed_exp2.values  # Converti il DataFrame in ndarray

In [ ]:
#TRAIN
random_forest_model_exp2 = oversampling_binary.train_random_forest(
    X_train_resampled_array_exp2, 
    y_train_resampled_exp2,
    X_val_imputed_array_exp2,
    y_val_encoded_exp2,
    X_test_imputed_array_exp2,
    y_test_encoded_exp2
);


In [ ]:
# 1. Effettua le predizioni sul set di convalida
y_pred_val_exp2 = random_forest_model_exp2.predict(X_val_imputed_array_exp2)

# 2. Effettua le predizioni sul set di test
y_pred_test_exp2 = random_forest_model_exp2.predict(X_test_imputed_array_exp2)

# 3. Calcola le matrici di confusione per convalida e test
cm_val_exp2 = confusion_matrix(y_val_encoded_exp2, y_pred_val_exp2)
cm_test_exp2 = confusion_matrix(y_test_encoded_exp2, y_pred_test_exp2)

# 4. Visualizza entrambe le matrici di confusione
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Matrice di confusione per il set di convalida
sns.heatmap(cm_val_exp2, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[0])
axes[0].set_title('Random Forest - Matrice di Confusione (Validazione)')
axes[0].set_xlabel('Predizioni')
axes[0].set_ylabel('Label')

# Matrice di confusione per il set di test
sns.heatmap(cm_test_exp2, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[1])
axes[1].set_title('Random Forest - Matrice di Confusione (Test)')
axes[1].set_xlabel('Predizioni')
axes[1].set_ylabel('Label')


plt.tight_layout()
plt.show()

In [ ]:
svm_model_exp2 = oversampling_binary.train_svm(X_train_resampled_array_exp2, y_train_resampled_exp2, X_val_imputed_array_exp2, y_val_encoded_exp2, X_test_imputed_array_exp2, y_test_encoded_exp2)

In [ ]:
# 1. Effettua le predizioni sul set di convalida
y_pred_val_exp2 = svm_model_exp2.predict(X_val_imputed_array_exp2)

# 2. Effettua le predizioni sul set di test
y_pred_test_exp2 = svm_model_exp2.predict(X_test_imputed_array_exp2)

# 3. Calcola le matrici di confusione per convalida e test
cm_val_exp2 = confusion_matrix(y_val_encoded_exp2, y_pred_val_exp2)
cm_test_exp2 = confusion_matrix(y_test_encoded_exp2, y_pred_test_exp2)

# 4. Visualizza entrambe le matrici di confusione
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Matrice di confusione per il set di convalida
sns.heatmap(cm_val_exp2, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[0])
axes[0].set_title('SVM - Matrice di Confusione (Validazione)')
axes[0].set_xlabel('Predizioni')
axes[0].set_ylabel('Label')

# Matrice di confusione per il set di test
sns.heatmap(cm_test_exp2, annot=True, fmt='d', cmap='Blues', xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'], ax=axes[1])
axes[1].set_title('SVM - Matrice di Confusione (Test)')
axes[1].set_xlabel('Predizioni')
axes[1].set_ylabel('Label')


plt.tight_layout()
plt.show()

In [ ]:
y_train_resampled_exp2 = y_train_resampled_exp2.astype(int)
y_val_encoded_exp2 = y_val_encoded_exp2.astype(int)
y_test_encoded_exp2 = y_test_encoded_exp2.astype(int)

In [ ]:
lightgbm_model_exp2 = oversampling_binary.train_lightgbm(X_train_resampled_array_exp2, y_train_resampled_exp2, X_val_imputed_array_exp2, y_val_encoded_exp2, X_test_imputed_array_exp2, y_test_encoded_exp2)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred_proba_exp2 = lightgbm_model_exp2.predict(X_val_imputed_array_exp2)
# Converti le probabilità in etichette di classe
y_val_pred_exp2 = np.argmax(y_val_pred_proba_exp2, axis=1)

# Predizioni del modello per il set di test
y_test_pred_proba_exp2 = lightgbm_model_exp2.predict(X_test_imputed_array_exp2)
# Converti le probabilità in etichette di classe
y_test_pred_exp2 = np.argmax(y_test_pred_proba_exp2, axis=1)

# Genera la matrice di confusione per il set di validazione
cm_val_exp2 = confusion_matrix(y_val_encoded_exp2, y_val_pred_exp2, labels=np.unique(y_val_encoded_exp2))
disp_val_exp2 = ConfusionMatrixDisplay(confusion_matrix=cm_val_exp2, display_labels=np.unique(y_val_encoded_exp2))

# Genera la matrice di confusione per il set di test
cm_test_exp2 = confusion_matrix(y_test_encoded_exp2, y_test_pred_exp2, labels=np.unique(y_test_encoded_exp2))
disp_test_exp2 = ConfusionMatrixDisplay(confusion_matrix=cm_test_exp2, display_labels=np.unique(y_test_encoded_exp2))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("LightGBM - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("LightGBM - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()